In [60]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imutils
from skimage.metrics import structural_similarity as ssim

from PIL import Image, ImageTk, ImageOps
from timeit import default_timer as timer

from xlwt import Workbook
import pandas as pd
from pathlib import Path

In [61]:
# NOISE REDUCTION
def noise_reduction(image) :
    im = cv2.imread(image)
    start = timer()
    yuv = cv2.cvtColor(im, cv2.COLOR_BGR2YCrCb) # or COLOR_BGR2YUV, should work equivalently here
    mean = np.mean(yuv, axis=(0,1))
    std = np.std(yuv, axis=(0,1))
    # EDIT: there needs to be an abs() on that difference, we want magnitudes, no signs
    mask = (np.abs(yuv - mean) / std >= 4.5).any(axis=2)
    mask_u8 = mask.astype(np.uint8) * 255
    end = timer()
    timeDiff = end - start

    print("noise_reduction for one image done!")
    return mask_u8, timeDiff

In [62]:
# Workbook is created
wb = Workbook()

def find_defects(imB, imA, k):
    imageB = cv2.imread(imB)
    imageA = cv2.imread(imA)
    start = timer()
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

    (score, diff) = ssim(grayA, grayB, full=True)

    # convert the array to 8-bit unsigned integers in the range [0,255] before we can use it with OpenCV
    diff = (diff * 255).astype("uint8")

    # threshold the difference image, followed by finding contours to obtain the regions of the two input images that differ
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)

    # find largest area contour
    max_area = -1
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if area>max_area:
            cnt = contours[i]
            max_area = area
    end = timer()
    timeDiff = end - start

#    print(max_area)
    sheets = ['image1', 'image2', 'image3', 'image4', 'image5', 'image6', 'image7', 'image8', 'image9', 'image10',
              'image11', 'image12', 'image13', 'image14', 'image15', 'image16', 'image17', 'image18', 'image19', 'image20',
              'image21', 'image22', 'image23', 'image24', 'image25', 'image26', 'image27', 'image28', 'image29', 'image30',
              'image31', 'image32', 'image33', 'image34', 'image35', 'image36', 'image37', 'image38', 'image39', 'image40',
              'image41', 'image42', 'image43', 'image44', 'image45', 'image46', 'image47', 'image48', 'image49', 'image50']
    tempSheet = wb.add_sheet(sheets[k])
    tempSheet.write(0, 0, 'x')
    tempSheet.write(0, 1, 'y')
    tempSheet.write(0, 2, 'w')
    tempSheet.write(0, 3, 'h')
    i,tw,ht=1,550,735
    # image = cv2.rectangle(image, start_point, end_point, color, thickness)
    # loop over the contours
    for c in contours:
        (x, y, w, h) = cv2.boundingRect(c)
        if w < tw and h < ht:
            cv2.rectangle(imageA, (x, y), (x + w, y + h), (0, 0, 255), 2) #original
            cv2.rectangle(imageB, (x, y), (x + w, y + h), (0, 0, 255), 2) #modified
            tempSheet.write(i, 0, x)
            tempSheet.write(i, 1, y)
            tempSheet.write(i, 2, w)
            tempSheet.write(i, 3, h)
            i +=1

    print("find_defects for one image done!")
    return imageB,timeDiff

In [ ]:
def find_defects(imB, imA, k):
    imageB = cv2.imread(imB)
    imageA = cv2.imread(imA)
    start = timer()
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

    (score, diff) = ssim(grayA, grayB, full=True)

    # convert the array to 8-bit unsigned integers in the range [0,255] before we can use it with OpenCV
    diff = (diff * 255).astype("uint8")

    # threshold the difference image, followed by finding contours to obtain the regions of the two input images that differ
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)

    # find largest area contour
    max_area = -1
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if area>max_area:
            cnt = contours[i]
            max_area = area
    end = timer()
    timeDiff = end - start

    tw,ht=550,750
    # image = cv2.rectangle(image, start_point, end_point, color, thickness)
    # loop over the contours
    for c in contours:
        (x, y, w, h) = cv2.boundingRect(c)
        if w < tw and h < ht:
            cv2.rectangle(imageA, (x, y), (x + w, y + h), (0, 0, 255), 2) #original
            cv2.rectangle(imageB, (x, y), (x + w, y + h), (0, 0, 255), 2) #modified

    return imageB,timeDiff

In [63]:
# Workbook is created
wb2 = Workbook()
tsheet = wb2.add_sheet('outputs')

def apply_algorithms():
    pathi = Path('dataset')
    patho = Path('outputs/noiseRedOut')

    inp = ['1r.tif', '2r.tif', '3r.tif', '4r.tif', '5r.tif', '6r.tif', '7r.tif', '8r.tif', '9r.tif', '10r.tif', 
            '11r.tif', '12r.tif', '13r.tif', '14r.tif', '15r.tif', '16r.tif', '17r.tif', '18r.tif', '19r.tif', '20r.tif', 
            '21r.tif', '22r.tif', '23r.tif', '24r.tif', '25r.tif', '26r.tif', '27r.tif', '28r.tif', '29r.tif', '30r.tif', 
            '31r.tif', '32r.tif', '33r.tif', '34r.tif', '35r.tif', '36r.tif', '37r.tif', '38r.tif', '39r.tif', '40r.tif', 
            '41r.tif', '42r.tif', '43r.tif', '44r.tif', '45r.tif', '46r.tif', '47r.tif', '48r.tif', '49r.tif', '50r.tif']

    out = ['out1.png', 'out2.png', 'out3.png', 'out4.png', 'out5.png', 'out6.png', 'out7.png', 'out8.png', 'out9.png', 'out10.png', 
            'out11.png', 'out12.png', 'out13.png', 'out14.png', 'out15.png', 'out16.png', 'out17.png', 'out18.png', 'out19.png', 'out20.png', 
            'out21.png', 'out22.png', 'out23.png', 'out24.png', 'out25.png', 'out26.png', 'out27.png', 'out28.png', 'out29.png', 'out30.png', 
            'out31.png', 'out32.png', 'out33.png', 'out34.png', 'out35.png', 'out36.png', 'out37.png', 'out38.png', 'out39.png', 'out40.png', 
            'out41.png', 'out42.png', 'out43.png', 'out44.png', 'out45.png', 'out46.png', 'out47.png', 'out48.png', 'out49.png', 'out50.png']

    tempPath = ""
    inputs1 = list()
    outputs1 = list()

    # for making lists of paths for cad images and deflectometry images
    for i in inp:
        tempPath = os.path.join(pathi, i).replace("\\","/")
        inputs1.append(tempPath)

    for o in out:
        tempPath = os.path.join(patho, o).replace("\\","/")
        outputs1.append(tempPath)

#    print(inputs1)
#    print(outputs1)

    # noise reduction
    timeDiff1=0
    time1=[] # nose_reduction time
    for imgName,outName in zip(inputs1,outputs1):
        returnImage, timeDiff1 = noise_reduction(imgName)
        cv2.imwrite(outName ,returnImage)
        time1.append(timeDiff1)

    pathi2 = Path('outputs/noiseRedOut')
    patho2 = Path('outputs/defectDetection')

    inputs2 = list()
    inputsCAD = list()
    outputs2 = list()
    # for making lists of paths for cad images and noise reduced images
    for i in out:
        tempPath = os.path.join(pathi2, i).replace("\\","/")
        inputs2.append(tempPath)

    for i in out:
        tempPath = os.path.join('cad', i).replace("\\","/")
        inputsCAD.append(tempPath)


    for o in out:
        tempPath = os.path.join(patho2, o).replace("\\","/")
        outputs2.append(tempPath)

  #  wb = Workbook()
    k=0
    timeDiff2=0
    time2 = [] # find defects time

    for in1,in2,out in zip(inputs2,inputsCAD,outputs2):
        returnImage,timeDiff2 = find_defects(in1,in2,k)
        cv2.imwrite(out ,returnImage)
        k+=1
        time2.append(timeDiff2)

    wb.save('infos.xls')

    total_time = [] # total time = noise reduction time + find defects time

    # add two list 
    for i in range(0, len(time1)):
        total_time.append(time1[i] + time2[i])
    print(total_time)

    tsheet.write(0, 0, 'image number')
    tsheet.write(0, 1, 'total time')
    tsheet.write(0, 2, 'f1 score')
    tsheet.write(0, 3, 'accuracy')

    for i in range(0,50):    
        tsheet.write(i+1, 0, i+1)
        tsheet.write(i+1, 1, total_time[i])

    wb2.save('output_scores.xls')
    print("\napply algorithms function done!")

In [64]:
apply_algorithms()

noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one image done!
noise_reduction for one imag

In [65]:
def calculate_performance():
    df1=pd.read_excel('infos.xls')
    df2=pd.read_excel('true_infos.xls')

    # put all the informations into lists
    df1 = pd.read_excel('infos.xls', sheet_name=0) # can also index sheet by name or fetch all sheets
    xt = df1['x'].tolist()
    yt = df1['y'].tolist()
    wt = df1['w'].tolist()
    ht = df1['h'].tolist()
    x,y,w,h = [],[],[],[]

    for i in range(0,len(xt)):
        x.append([xt[i]])
        y.append([yt[i]])
        w.append([wt[i]])
        h.append([ht[i]])

    for i in range(0,len(x)):
        x[i].extend(y[i])
        x[i].extend(w[i])
        x[i].extend(h[i])

    #a = [x,y,w,h]

    TP, FP, TN, FN = 0,0,0,0
    score_f1 = []
    score_accuracy = []

    for sheet in range(0,50):
        df1 = pd.read_excel('infos.xls', sheet_name=sheet)
        df2 = pd.read_excel('true_infos.xls', sheet_name=sheet)

        xt1 = df1['x'].tolist()
        yt1 = df1['y'].tolist()
        wt1 = df1['w'].tolist()
        ht1 = df1['h'].tolist()
        x1,y1,w1,h1 = [],[],[],[]

        for i in range(0,len(xt1)):
            x1.append([xt1[i]])
            y1.append([yt1[i]])
            w1.append([wt1[i]])
            h1.append([ht1[i]])

        for i in range(0,len(x1)):
            x1[i].extend(y1[i])
            x1[i].extend(w1[i])
            x1[i].extend(h1[i])
        ##
        xt2 = df2['x'].tolist()
        yt2 = df2['y'].tolist()
        wt2 = df2['w'].tolist()
        ht2 = df2['h'].tolist()
        x2,y2,w2,h2 = [],[],[],[]

        for i in range(0,len(xt2)):
            x2.append([xt2[i]])
            y2.append([yt2[i]])
            w2.append([wt2[i]])
            h2.append([ht2[i]])

        for i in range(0,len(x2)):
            x2[i].extend(y2[i])
            x2[i].extend(w2[i])
            x2[i].extend(h2[i])

        infos = x1;
        true_infos = x2;

        print("\tIMAGE", sheet+1)

        ## comparison

        tp,fp,tn,fn = 0,0,0,0

        for i in range(0,len(infos)):
            temp = infos[i]
            if temp in true_infos:
                TP+=1
                tp+=1
            else:
                FP+=1
                fp+=1

        for i in range(0,len(true_infos)):
            temp = true_infos[i]
            if temp not in infos:
                FN +=1  # in true_info but not in info
                fn +=1

        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        F1 = (2 * precision * recall) / (precision + recall)
        accuracy = ((tp + tn) / (tp + tn + fp +fn) ) * 100
        score_f1.append(F1)
        score_accuracy.append(accuracy)

        print("TP\t:",tp)
        print("FP\t:",fp)
        print("FN\t:",fn)
        print("F1 score\t: %5.2f" % F1)
        print("Accuracy\t: %5.2f" % accuracy, "%")

    average_f1 = sum(score_f1) / len(score_f1)
    print("\n\nAverage F1 Score %5.2f" % average_f1)

    average_acc = sum(score_accuracy) / len(score_accuracy)
    print("\nAverage Accuracy %5.2f" % average_acc)

    for i in range(0,50):
        tsheet.write(i+1, 2, score_f1[i])
        tsheet.write(i+1, 3, score_accuracy[i])

    wb2.save('output_scores.xls')
    print("\ncalculate performance function done!")

In [66]:
calculate_performance()

	IMAGE 1
TP	: 39
FP	: 16
FN	: 0
F1 score	:  0.83
Accuracy	: 70.91 %
	IMAGE 2
TP	: 18
FP	: 1
FN	: 0
F1 score	:  0.97
Accuracy	: 94.74 %
	IMAGE 3
TP	: 9
FP	: 1
FN	: 0
F1 score	:  0.95
Accuracy	: 90.00 %
	IMAGE 4
TP	: 21
FP	: 16
FN	: 0
F1 score	:  0.72
Accuracy	: 56.76 %
	IMAGE 5
TP	: 101
FP	: 30
FN	: 8
F1 score	:  0.84
Accuracy	: 72.66 %
	IMAGE 6
TP	: 47
FP	: 2
FN	: 0
F1 score	:  0.98
Accuracy	: 95.92 %
	IMAGE 7
TP	: 43
FP	: 3
FN	: 2
F1 score	:  0.95
Accuracy	: 89.58 %
	IMAGE 8
TP	: 5
FP	: 0
FN	: 0
F1 score	:  1.00
Accuracy	: 100.00 %
	IMAGE 9
TP	: 6
FP	: 8
FN	: 1
F1 score	:  0.57
Accuracy	: 40.00 %
	IMAGE 10
TP	: 6
FP	: 0
FN	: 0
F1 score	:  1.00
Accuracy	: 100.00 %
	IMAGE 11
TP	: 24
FP	: 7
FN	: 0
F1 score	:  0.87
Accuracy	: 77.42 %
	IMAGE 12
TP	: 48
FP	: 1
FN	: 1
F1 score	:  0.98
Accuracy	: 96.00 %
	IMAGE 13
TP	: 334
FP	: 9
FN	: 4
F1 score	:  0.98
Accuracy	: 96.25 %
	IMAGE 14
TP	: 119
FP	: 3
FN	: 5
F1 score	:  0.97
Accuracy	: 93.70 %
	IMAGE 15
TP	: 80
FP	: 1
FN	: 12
F1 score	:  0.92
Acc